In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [30]:
df = pd.read_csv('../data/raw.csv')

In [31]:
df

,Age,Gender,Diagnosis,Symptom Severity (1-10),Mood Score (1-10),Sleep Quality (1-10),Physical Activity (hrs/week),Stress Level (1-10)
0,43,Female,Major Depressive Disorder,10,5,8,5,9
1,40,Female,Major Depressive Disorder,9,5,4,7,7
2,55,Female,Major Depressive Disorder,6,3,4,3,7
3,34,Female,Major Depressive Disorder,6,3,6,5,8
4,52,Male,Panic Disorder,7,6,6,8,5
...,...,...,...,...,...,...,...,...
495,24,Male,Generalized Anxiety,10,4,8,6,9
496,22,Male,Panic Disorder,5,6,6,7,6
497,23,Male,Major Depressive Disorder,7,3,4,2,5
498,48,Male,Bipolar Disorder,9,4,6,9,6


In [32]:
X =df.drop(columns=['Diagnosis'],axis=1)
y=df['Diagnosis']

In [33]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
# Now one by one we combine them in a pipeline 
# 1st step in One Hot Encoding
# 2nd step is Standardization
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y= le.fit_transform(y)


In [35]:
 X = preprocessor.fit_transform(X)


In [36]:
X.shape

(500, 8)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((400, 8), (100, 8))

In [38]:
y_train

array([2, 0, 0, 2, 0, 3, 1, 1, 2, 3, 1, 1, 0, 0, 2, 2, 2, 0, 0, 3, 1, 2,
       0, 3, 1, 1, 0, 2, 0, 1, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 1, 2, 0,
       0, 2, 3, 3, 2, 3, 3, 0, 0, 0, 0, 2, 2, 0, 3, 1, 1, 0, 0, 2, 0, 0,
       0, 2, 2, 3, 1, 1, 0, 0, 1, 3, 2, 2, 2, 3, 1, 0, 2, 0, 1, 1, 2, 2,
       0, 3, 3, 1, 1, 1, 0, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 0, 0,
       0, 0, 3, 2, 2, 3, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 3, 3, 0, 0, 1, 2,
       3, 1, 2, 3, 3, 2, 3, 2, 3, 3, 0, 3, 3, 2, 2, 2, 3, 2, 3, 1, 1, 2,
       1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 2, 2, 3, 0, 1, 0,
       1, 3, 1, 1, 1, 0, 1, 2, 1, 0, 0, 2, 1, 3, 2, 0, 0, 1, 0, 1, 3, 3,
       2, 1, 2, 0, 3, 0, 3, 3, 3, 0, 0, 2, 2, 3, 0, 2, 2, 1, 1, 1, 0, 0,
       2, 0, 3, 2, 0, 0, 3, 3, 1, 1, 1, 1, 0, 2, 2, 1, 2, 0, 2, 2, 3, 0,
       0, 3, 3, 3, 0, 2, 0, 2, 3, 1, 3, 1, 1, 1, 2, 3, 2, 0, 3, 1, 1, 1,
       1, 1, 3, 2, 3, 3, 1, 0, 3, 1, 0, 3, 1, 2, 0, 2, 0, 3, 2, 0, 3, 1,
       1, 3, 3, 1, 2, 0, 1, 3, 3, 0, 2, 0, 0, 3, 0,

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_classification(y_true, y_pred):
    """
    Evaluate classification performance using common metrics.

    Parameters:
    -----------
    y_true : array-like
        True class labels
    y_pred : array-like
        Predicted class labels

    Returns:
    --------
    accuracy : float
    precision : float
    recall : float
    f1 : float
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    return accuracy, precision, recall, f1


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [41]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "CatBoost Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier()
}

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_classification(y_true, y_pred):
    """
    Evaluate classification performance using Accuracy, Precision, Recall, and F1.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    return accuracy, precision, recall, f1


# -------------------- MODEL LOOP -------------------- #
model_list = []
f1_list = []

for name, model in models.items():
    # Train model
    model.fit(X_train, y_train)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate with function
    train_acc, train_prec, train_rec, train_f1 = evaluate_classification(y_train, y_train_pred)
    test_acc, test_prec, test_rec, test_f1 = evaluate_classification(y_test, y_test_pred)

    # Print results
    print(f"Model: {name}")
    print("Training Performance")
    print(f"- Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_rec:.4f}, F1: {train_f1:.4f}")
    print("----------------------------------")
    print("Test Performance")
    print(f"- Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}")
    print("="*40, "\n")

    # Store for best model selection
    model_list.append(name)
    f1_list.append(test_f1)

# -------------------- BEST MODEL -------------------- #
best_index = f1_list.index(max(f1_list))
print(f"✅ Best Model: {model_list[best_index]} with Test F1 Score: {f1_list[best_index]:.4f}")


Model: Logistic Regression
Training Performance
- Accuracy: 0.3150, Precision: 0.3160, Recall: 0.3150, F1: 0.3089
----------------------------------
Test Performance
- Accuracy: 0.3100, Precision: 0.2901, Recall: 0.3100, F1: 0.2918

Model: K-Neighbors Classifier
Training Performance
- Accuracy: 0.4825, Precision: 0.4989, Recall: 0.4825, F1: 0.4791
----------------------------------
Test Performance
- Accuracy: 0.2100, Precision: 0.2204, Recall: 0.2100, F1: 0.2123

Model: Decision Tree
Training Performance
- Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
----------------------------------
Test Performance
- Accuracy: 0.2800, Precision: 0.2831, Recall: 0.2800, F1: 0.2809

Model: Random Forest Classifier
Training Performance
- Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
----------------------------------
Test Performance
- Accuracy: 0.2200, Precision: 0.2209, Recall: 0.2200, F1: 0.2202



c:\Users\Tridib Roy Chowdhury\OneDrive\Desktop\CODING\SIH_prototype\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:01:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model: XGBClassifier
Training Performance
- Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
----------------------------------
Test Performance
- Accuracy: 0.2100, Precision: 0.2077, Recall: 0.2100, F1: 0.2083

Model: CatBoost Classifier
Training Performance
- Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
----------------------------------
Test Performance
- Accuracy: 0.2400, Precision: 0.2470, Recall: 0.2400, F1: 0.2425

Model: AdaBoost Classifier
Training Performance
- Accuracy: 0.3775, Precision: 0.3745, Recall: 0.3775, F1: 0.3718
----------------------------------
Test Performance
- Accuracy: 0.3200, Precision: 0.2833, Recall: 0.3200, F1: 0.2930

✅ Best Model: AdaBoost Classifier with Test F1 Score: 0.2930


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Define parameter grids for each model
 

# Dictionary of models
models = {
    "RandomForestClassifier": RandomForestClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "CatBoostClassifier": CatBoostClassifier(verbose=False)
}

best_models = {}

# Run GridSearchCV for each model
for name, model in models.items():
    print(f"🔎 Tuning {name}...")
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        scoring='f1_weighted',   # since classes might be imbalanced
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)
    
    print(f"✅ Best Params for {name}: {grid.best_params_}")
    print(f"✅ Best F1 Score (CV): {grid.best_score_:.4f}")
    
    best_models[name] = grid.best_estimator_

# Pick best overall
best_model_name = max(best_models, key=lambda x: best_models[x].score(X_test, y_test))
print(f"\n🏆 Best Overall Model: {best_model_name}")


🔎 Tuning RandomForestClassifier...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
✅ Best Params for RandomForestClassifier: {'class_weight': 'balanced', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
✅ Best F1 Score (CV): 0.2739
🔎 Tuning AdaBoostClassifier...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
✅ Best Params for AdaBoostClassifier: {'learning_rate': 1, 'n_estimators': 50}
✅ Best F1 Score (CV): 0.2727
🔎 Tuning XGBClassifier...
Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Users\Tridib Roy Chowdhury\OneDrive\Desktop\CODING\SIH_prototype\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:02:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Best Params for XGBClassifier: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
✅ Best F1 Score (CV): 0.2742
🔎 Tuning CatBoostClassifier...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
✅ Best Params for CatBoostClassifier: {'depth': 4, 'iterations': 500, 'learning_rate': 0.1}
✅ Best F1 Score (CV): 0.2715

🏆 Best Overall Model: AdaBoostClassifier


In [44]:
best_models

{'RandomForestClassifier': RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=200),
 'AdaBoostClassifier': AdaBoostClassifier(learning_rate=1),
 'XGBClassifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='mlogloss',
               feature_types=None, feature_weights=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=3, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=100, n_jobs=None,
               num_parallel_tree=None, ...),
 'CatBoostClassifier': <cat

In [45]:
from sklearn.ensemble import AdaBoostClassifier

# Initialize AdaBoost with best parameters (if you did hyperparameter tuning)
best_adaboost = AdaBoostClassifier(
    n_estimators=100,      # example, use tuned value
    learning_rate=1.0,     # example, use tuned value
    random_state=42
)

# Fit the model on training data
best_adaboost.fit(X_train, y_train)

,estimator,None
,n_estimators,100
,learning_rate,1.0
,algorithm,'deprecated'
,random_state,42


In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Accuracy
test_acc = accuracy_score(y_test, y_test_pred)
# Precision, Recall, F1
test_prec = precision_score(y_test, y_test_pred, average='weighted')
test_rec = recall_score(y_test, y_test_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

print("AdaBoost Test Performance")
print(f"- Accuracy: {test_acc:.4f}")
print(f"- Precision: {test_prec:.4f}")
print(f"- Recall: {test_rec:.4f}")
print(f"- F1 Score: {test_f1:.4f}")

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


AdaBoost Test Performance
- Accuracy: 0.3200
- Precision: 0.2833
- Recall: 0.3200
- F1 Score: 0.2930

Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.12      0.15        24
           1       0.39      0.59      0.47        29
           2       0.40      0.42      0.41        24
           3       0.14      0.09      0.11        23

    accuracy                           0.32       100
   macro avg       0.28      0.30      0.28       100
weighted avg       0.28      0.32      0.29       100



In [47]:
new_data = pd.DataFrame([
    {
        "Age": 15,
        "Gender": "Male",
        "Sleep Quality (1-10)": 3,
        "Stress Level (1-10)": 7,
        "Symptom Severity (1-10)": 10,
        "Mood Score (1-10)": 8,
        "Physical Activity (hrs/week)": 4
    },
])

In [48]:
# Preprocess new data exactly like your training data
# If you used a preprocessor like ColumnTransformer or OneHotEncoder, apply it here
X_new = preprocessor.transform(new_data)  # DO NOT use fit_transform, only transform

# Predict using the best AdaBoost model
y_new_pred = best_models['AdaBoostClassifier'].predict(X_new)
y_new_pred_real = le.inverse_transform(y_new_pred)
print(y_new_pred_real)

['Panic Disorder']


In [52]:
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

In [49]:
import pickle
pickle.dump(preprocessor,open('preprocessor.pkl','wb'))


In [50]:
import pickle

# Save the model
with open("adaboost_model.pkl", "wb") as f:
    pickle.dump(best_models['AdaBoostClassifier'], f)

print("✅ Model saved as 'adaboost_model.pkl'")


✅ Model saved as 'adaboost_model.pkl'
